In [1]:
import mne
import os
# os.chdir('C:\\Users\\Esimk_000\\Desktop\\School\\Spring 2018\\Cs59866\\EEG Project\\seniorprojecteeg-master-fork\\preprocessing\\Artifact_Removal')
%run preprocecssing_helpers.ipynb
%run Batch_ArtifactFilter_Epoch.ipynb
%run ../StimCodes.ipynb

import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)
matplotlib.rcParams.update({'font.size': 15})

In [84]:
# path = 'D:/EEGLAB/'
# eeglab_files = list_eeglab_files(path)

path = '/sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/'

In [85]:
files = os.listdir(path)

In [86]:
epoch_files = [x for x in files if '.fif' in x]

In [87]:
epoch_files

['20140320_1421_epo.fif',
 '20140313_1439_epo.fif',
 '20140206_1029_epo.fif',
 '20140402_1603_epo.fif',
 '20131216_1441_epo.fif',
 '20140318_1043_epo.fif',
 '20140304_1418_epo.fif',
 '20140319_1043_epo.fif',
 '20140331_1753_epo.fif',
 '20140418_1421_epo.fif',
 '20140217_1211_epo.fif',
 '20140219_1046_epo.fif',
 '20140306_1133_epo.fif',
 '20140324_1553_epo.fif',
 '20140225_1422_epo.fif',
 '20140415_1309_epo.fif',
 '20140402_1734_epo.fif',
 '20140411_1105_epo.fif',
 '20140220_1123_epo.fif',
 '20140410_1217_epo.fif',
 '20140227_1410_epo.fif',
 '20140227_1058_epo.fif',
 '20140325_1007_epo.fif',
 '20140205_1114_epo.fif',
 '20140206_1151_epo.fif',
 '20140318_1211_epo.fif',
 '20140404_1418_epo.fif',
 '20140314_1553_epo.fif',
 '20140418_1546_epo.fif',
 '20140207_1146_epo.fif',
 '20140219_1205_epo.fif',
 '20140207_1219_epo.fif',
 '20140324_1441_epo.fif',
 '20140327_1022_epo.fif',
 '20140205_1230_epo.fif',
 '20131216_1310_epo.fif',
 '20140411_0929_epo.fif',
 '20140313_1630_epo.fif',
 '20140410_1

In [96]:
epoch_file = os.path.join(path, epoch_files[10])

In [97]:
epoch = mne.read_epochs(epoch_file, preload = True)

Reading /sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/20140217_1211_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1859 matching events found
Applying baseline correction (mode: mean)
1859 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [8]:
# eeglab_files

In [9]:
# raw = mne.io.read_raw_eeglab(eeglab_files[4],preload = True)

In [10]:
# os.chdir("F:\\")
# epoch.save(eeglab_files[4].split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)

In [11]:
# events = find_events(raw)
# eeglab_trials = get_trials(events)
# eeglab_trials[1:3]

In [12]:
modality_lexicality_event_ids = {
   "AALL" : 112,
   "AALN" : 212,
   "AANL" : 312,
   "AANN" : 412,
   "AVLL" : 512,
   "AVNN" : 612,
   "VALL" : 712,
   "VANN" : 812,
   "VVLL" : 912,
   "VVLN" : 1012,
   "VVNL" : 1112,
   "VVNN" : 1212
}

In [13]:
def get_epoch_trials(epoch_object):
    #note: third stim-code has a value that is 1 more than the first.
    events = epoch_object.events
    found_first_stim_code = False
    found_second_stim_code = False
    found_third_stim_code = False
    current_trial = []
    trials = []
    for event in events:
        if not found_first_stim_code and not found_second_stim_code and not found_third_stim_code: #still searching for the end of trial
            current_trial.append(event)
            found_first_stim_code = True
            first_stim_code = event
        elif not found_second_stim_code and not found_third_stim_code:
            current_trial.append(event)
            found_second_stim_code = True
        elif not found_third_stim_code and (event[-1] + 1) == first_stim_code[-1]: 
            current_trial.append(event)
            found_third_stim_code = True
            trials.append(current_trial)
            current_trial = []
            found_first_stim_code = False
            found_second_stim_code = False
            found_third_stim_code = False
        elif not found_third_stim_code and (event[-1] + 1) != first_stim_code[-1]:
            # there is no third event, trial is unfinished.
            current_trial = []
            current_trial.append(event)#assuming it's first stim-code...for now
            found_first_stim_code = True
    return trials

In [14]:
def get_stim_code_epoch_trials(epoch_object):
    events = epoch_object.events
    found_first_stim_code = False
    found_second_stim_code = False
    found_third_stim_code = False
    current_trial = []
    trials = []
    
    for i in range(len(events[:,-1])-2):
        found_third_stim_code = False
        while (not found_third_stim_code):
            stim_code_1 = events[i]
            stim_code_2 = events[i+1]
            stim_code_3 = events[i+2]
#             print(stim_code_1,stim_code_2,stim_code_3)
            
            if stim_code_1[-1] == stim_code_3[-1] + 1:
                trials.append([stim_code_1,stim_code_2,stim_code_3])
                found_third_stim_code = True
            else:
                
                break;
    return trials

In [15]:
def stim_code_combination_from_trial(stim_code_tuple, stim_combinations):
    """
    Description:
        returns the correct stim-code combination event id 
        example - a trial has events (20, 15, 19), this method will match those values to a 
        stim combination that is defined in the dictionary stim_combinations
    """
    for stim_code in stim_combinations.keys():
        if stim_code_tuple == stim_code:
            return stim_combinations[stim_code]
    return "NaN"

In [16]:
def convert_epoch_events_to_stim_combinations(epoch_object):
    trials = get_stim_code_epoch_trials(epoch_object)
#     print(trials)
    new_events_master = []
    
    for trial in trials:
#         print(trial)
        trial = np.array(trial)
        stim_codes = trial[:,-1]
        try:
#             print("stim_codes",stim_codes)
            stim_code_tuple = (stim_codes[0],stim_codes[1], stim_codes[2])
#             print("stim_code_tuple",stim_code_tuple)
            stim_combination = stim_code_combination_from_trial(stim_code_tuple, stim_combinations)
#             print("stim_combination",stim_combination)
            new_event_id = modality_lexicality_event_ids[stim_combination]
#             print("New event_id", new_event_id)
            trial[:,-1] = new_event_id
            new_events_master.append(trial)
        except:
            print("Error with stim-code: {0}".format(stim_code_tuple))
#     print(new_events_master)
    return np.concatenate([x for x in new_events_master])

In [98]:
new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch.copy())
new_events.shape

Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (24, 22, 23)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim

(1857, 3)

In [99]:
print(new_events.shape)
print(epoch.events.shape)

(1857, 3)
(1859, 3)


In [22]:
checked_events = np.array([x for x in new_events if x[0] in epoch.events[:,0]])

In [28]:
def epoch_indexes_to_drop(epoch_object, events):
    epoch_events = epoch_object.events
    indicies_to_drop = []
    for i in range(len(epoch_events)):
        if epoch_events[i,0] not in events[:,0]:
            indicies_to_drop.append(i)
    return indicies_to_drop

In [144]:
import itertools
def generate_indicies_tuples(indicies_to_drop, len_epochs):
    indicides_tuples = []
    if indicies_to_drop !=[]:
        
        for i in range(len(indicies_to_drop)):
            if i == 0:
                print("i ==0")
                indicides_tuples.append((0,indicies_to_drop[i]))
                print(indicides_tuples)
            else:
                if indicies_to_drop[i-1] +1 == indicies_to_drop[i]:
                    pass
    #                 indicides_tuples.append((indicies_to_drop[i-1] +1, indicies_to_drop[i] +1 ))

                else:
                    indicides_tuples.append((indicies_to_drop[i-1] +1, indicies_to_drop[i]))
            print(indicies_to_drop)

        indicides_tuples.append((indicies_to_drop[-1] +1, len_epochs))
        return indicides_tuples
    else:
        return []
        
def drop_bad_epoch_events(epoch_object, indicies_to_drop):
    indicies_tuples = generate_indicies_tuples(indicies_to_drop, epoch.events.shape[0])
    print("indicides tuple", indicies_tuples)
    epoch_list = []
    if indicies_tuples !=[]:
        for index in indicies_tuples:
            epoch_list.append(epoch_object[index[0]:index[1]])
        if epoch_list != []:
            return mne.epochs.concatenate_epochs(epoch_list)
    else: 
        return epoch_object
#     else:
#         return epoch_object

In [145]:
for epoch_file in epoch_files:
    file = os.path.join(path, epoch_file)
    epoch = mne.read_epochs(file, preload = True);
    
    print("**epoch events shape", epoch.events.shape)
#     try:
        
    new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch.copy())
    print("**new_events shape", new_events.shape)

    indicies_to_drop = epoch_indexes_to_drop(epoch.copy(), new_events)
    print("**indicies to drop", indicies_to_drop)
    new_epoch = drop_bad_epoch_events(epoch.copy(), indicies_to_drop=indicies_to_drop)
    print("**new epoch shape", new_epoch.get_data().shape)
#     except:
#         print("**Error with file {0}".format(epoch_file))

Reading /sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/20140320_1421_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
2041 matching events found
Applying baseline correction (mode: mean)
2041 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
**epoch events shape (2041, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
E

Error with stim-code: (21, 19, 20)
Error with stim-code: (21, 19, 20)
Error with stim-code: (21, 19, 20)
Error with stim-code: (7, 11, 6)
**new_events shape (2037, 3)
**indicies to drop [597, 598, 599, 600]
i ==0
[(0, 597)]
[597, 598, 599, 600]
[597, 598, 599, 600]
[597, 598, 599, 600]
[597, 598, 599, 600]
indicides tuple [(0, 597), (601, 2041)]
2037 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 bad epochs dropped
**new epoch shape (2037, 129, 257)
Reading /sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/20140313_1439_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1913 matching events found
Applying baseline correction (mode: mean)
1913 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
**epoch events shape (1913, 3)
Error with stim-code: (6, 4, 5)
Error with stim-code: (6, 4, 5)
Error with stim-cod

**new_events shape (1908, 3)
**indicies to drop [837, 838, 839, 840, 1345]
i ==0
[(0, 837)]
[837, 838, 839, 840, 1345]
[837, 838, 839, 840, 1345]
[837, 838, 839, 840, 1345]
[837, 838, 839, 840, 1345]
[837, 838, 839, 840, 1345]
indicides tuple [(0, 837), (841, 1345), (1346, 1913)]
1908 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 bad epochs dropped
**new epoch shape (1908, 129, 257)
Reading /sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/20140206_1029_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1917 matching events found
Applying baseline correction (mode: mean)
1917 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
**epoch events shape (1917, 3)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with

**new_events shape (1917, 3)
**indicies to drop []
indicides tuple []
**new epoch shape (1917, 129, 257)
Reading /sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/20140402_1603_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1974 matching events found
Applying baseline correction (mode: mean)
1974 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
**epoch events shape (1974, 3)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-c

**new_events shape (1974, 3)
**indicies to drop []
indicides tuple []
**new epoch shape (1974, 129, 257)
Reading /sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/20131216_1441_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1753 matching events found
Applying baseline correction (mode: mean)
1753 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
**epoch events shape (1753, 3)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9, 7, 8)
Error with stim-code: (9

...

Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23, 21)
Error with stim-code: (22, 23,

Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (12, 10, 11)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Error with stim-code: (3, 1, 2)
Erro

Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (10, 11, 9)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with stim-code: (4, 5, 3)
Error with sti

Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (16, 17, 15)
Error with stim-code: (15, 16, 14)
Error with stim-code: (16, 14, 15)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code: (15, 13, 14)
Error with stim-code

**indicies to drop [1452, 1453, 1454, 1455]
i ==0
[(0, 1452)]
[1452, 1453, 1454, 1455]
[1452, 1453, 1454, 1455]
[1452, 1453, 1454, 1455]
[1452, 1453, 1454, 1455]
indicides tuple [(0, 1452), (1456, 2086)]
2082 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 bad epochs dropped
**new epoch shape (2082, 129, 257)


In [134]:
epoch.events[597:605]

array([[264897,      0,     14],
       [265147,      0,     15],
       [265150,      0,      1],
       [265389,      0,     13],
       [272744,      0,      2],
       [272994,      0,      6],
       [273235,      0,      1],
       [273857,      0,      2]])

In [119]:
epoch.events.shape[0]

1859

In [120]:
indicies_to_drop = epoch_indexes_to_drop(epoch.copy(), new_events)
new_epoch = drop_bad_epoch_events(epoch.copy(), indicies_to_drop=indicies_to_drop)

i ==0
[(0, 825)]
[825, 1579]
[825, 1579]
[(0, 825), (826, 1579), (1580, 1859)]
1857 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 bad epochs dropped


In [121]:
new_epoch.events.shape

(1857, 3)

In [122]:
epoch.events = new_events


In [34]:
new_epochs = mne.epochs.concatenate_epochs([epoch[:1116], epoch[1117:]])

1758 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 bad epochs dropped


In [37]:
new_epochs.events = new_events

In [39]:
new_epochs.events.shape

(1758, 3)

In [23]:
checked_events.shape

(1758, 3)

In [24]:
epoch.get_data().shape

(1759, 129, 257)

In [8]:
def isCorrect_stim_code_trials(trials):
    """
    Description:
        A sanity check that makes sure all trials are in the correct order
        The third stim-code needs to have a value that is one less than the 
        first stim-code.
        
    Variables:
        trials: An ndarray of shape  (n_trials, 3, 3)
    -----
    returns:
        boolean if all are correct, or one is incorrect. 
        It will print all incorrect trials
    """
    has_incorrect_trial = True
    for trial in trials:
        if trial[:,-1][0] == trial[:,-1][-1] + 1:
            pass
        else:
            print("Incorrect Trial \n: {0}".format(trial))
            has_incorrect_trial = False
            
                    
    return has_incorrect_trial

In [ ]:
# isCorrect_stim_code_trials(epoch_trials)

In [ ]:
# def create_describer_events(epoch, eeglab):
#     """
#     Description:
#         uses raw eeglab events and epoch events to return a new dictionary
#         with key the timestamp of an event, and value is the new event_id.
#         These event_ids can later be split/manipulated to label data by specific 
#         events
#     Variables:
#         epoch: mne.io.epoch object
#         raw_events: an ndarray of 3d integers shape 
        
#     returns:
#         dictionary
#     """
#     raw_events = find_events(eeglab)
#     raw_trials = get_trials(raw_events)
#     epoch_events = epoch.events
#     epoch_trials = get_epoch_trials(epoch)
    
#     for epoch_trial in epoch_trials:
#         for event in epoch_trial
#             if event[0] in 
#     #  

In [50]:
(14,18,13) == (14, 18, 13)

True